In [11]:
#Import libraries
import pandas as pd
import numpy as np
import plotly.graph_objects as go

#Read the cluster data
dfzc=pd.read_csv('resource/df_VAZc.csv',index_col=[0])
Fields=[
'region',
'period_begin',
'period_end',
'period_duration',
'parent_metro_region',
'property_type',
'median_sale_price',
'median_sale_price_yoy',
'homes_sold',
'new_listings_yoy',
'median_dom',
'median_dom_yoy',
'avg_sale_to_list',
'cluster_nbr',
'0',
'1',
'2'
]
#Only select fields that are needed in this visualization
df_VA=dfzc[Fields]
len(df_VA)

205

In [14]:
import glob

files = glob.glob('data/*.csv') 
df_list = []
df_all = pd.DataFrame([])
for f in files:
    df_list.append(f[3:-4])
    df = pd.read_csv(f,index_col=[0])
    df.insert(0,'group',f[8:-4])
    df_all = pd.concat([df_all, df])
df_all['group'].unique()

array(['county', 'neighborhood', 'zipcode'], dtype=object)

In [15]:
df_all[df_all['group']=='county'].head()


,group,period_begin,period_end,period_duration,region_type,region_type_id,table_id,is_seasonally_adjusted,region,city,...,sold_above_list_yoy,price_drops,price_drops_mom,price_drops_yoy,off_market_in_two_weeks,off_market_in_two_weeks_mom,off_market_in_two_weeks_yoy,parent_metro_region,parent_metro_region_metro_code,last_updated
6803,county,2021-07-01,2021-07-31,30,county,5,3002,f,"Orange County, VA",NaN,...,0.184982,0.169935,0.044935,-0.003718,0.414634,0.007227,0.090690,Virginia nonmetropolitan area,NaN,2022-01-09 14:29:56
16181,county,2021-07-01,2021-07-31,30,county,5,3042,f,"Fairfax City County, VA",NaN,...,0.166667,0.351351,0.163851,0.166166,0.545455,-0.061688,-0.061688,"Washington, DC",47894.0,2022-01-09 14:29:56
18476,county,2021-07-01,2021-07-31,30,county,5,2965,f,"Fairfax County, VA",NaN,...,0.155436,0.260093,0.006198,0.048141,0.448780,-0.138322,-0.034856,"Washington, DC",47894.0,2022-01-09 14:29:56
28014,county,2021-07-01,2021-07-31,30,county,5,3055,f,"Newport News, VA",NaN,...,0.276573,0.192192,0.072738,0.057272,0.433476,-0.060451,0.091371,"Virginia Beach, VA",47260.0,2022-01-09 14:29:56
32101,county,2021-07-01,2021-07-31,30,county,5,2978,f,"Hanover County, VA",NaN,...,0.220005,0.326923,0.048135,0.064685,0.652695,-0.012045,0.004210,"Richmond, VA",40060.0,2022-01-09 14:29:56


In [9]:
#Rename fields to make them easier to understand
df = df_VA.rename({'region': 'Zip_Code', '0': 'PC1','1':'PC2','2':'PC3'}, axis=1)
df['cluster_nbr'] = df['cluster_nbr'].apply(str) #Change data type for cluster_nbr

#Create summary statistics for clusters. Calucating medians for the following metrics at cluster level
cluster_group = ['homes_sold','median_sale_price','median_sale_price_yoy','new_listings_yoy','avg_sale_to_list','median_dom','median_dom_yoy']
df1=df.groupby(['cluster_nbr'])['cluster_nbr'].count()
df2=df.groupby(['cluster_nbr'])[cluster_group].median()
df_summary=pd.concat([df1, df2],axis=1)
df_summary = df_summary.rename({'cluster_nbr': '# of Zips','homes_sold':'Median Homes Sold','median_sale_price':'Median Sale Price','median_sale_price_yoy':'Median Sale Price YoY','new_listings_yoy':'Median New Listings YoY','avg_sale_to_list':'Median Sale-to-List Ratio','median_dom':'Median Days on Market','median_dom_yoy':'Median Days on Market YoY (days)'}, axis=1)
df_summary

,# of Zips,Median Homes Sold,Median Sale Price,Median Sale Price YoY,Median New Listings YoY,Median Sale-to-List Ratio,Median Days on Market,Median Days on Market YoY (days)
cluster_nbr,,,,,,,,
0,54,111.5,431250.0,0.140014,-0.024202,1.028527,9.00,-5.25
1,79,97.0,525000.0,0.079137,0.004630,1.014521,19.00,-6.50
2,44,88.0,310000.0,0.106040,0.423500,1.011687,18.00,-9.50
3,28,32.5,475000.0,0.144182,-0.108187,0.999658,33.75,-7.00


In [13]:
#Add cluster description to the dataframe
def f(row):
    if row['cluster_nbr'] == '0':
        val = 'Comment 1'
    elif row['cluster_nbr'] == '1':
        val = 'Comment 2'
    elif row['cluster_nbr'] == '2':
        val = 'Commet 3'
    elif row['cluster_nbr'] == '3':
        val = 'Comment 4'   
    # elif row['cluster_nbr'] == '4':
    #     val = 'Coment 5'  
    else:
        val = 'NA'
    return val

df['cluster_desc']= df.apply(f, axis=1)

In [14]:
#Define a function to assign a unique color code to each cluster. 
#You can choose any color code you want by tweaking the val parameter
def color(row):
    if row['cluster_nbr'] == '0':
        val = '#0984BD'
    elif row['cluster_nbr'] == '1':
        val = '#E12906'
    elif row['cluster_nbr'] == '2':
        val = '#08E9E7'
    elif row['cluster_nbr'] == '3':
        val = '#E18A06'   
    # elif row['cluster_nbr'] == '4':
    #     val = '#0C861A'  
    else:
        val = 'NA'
    return val
#Apply the function to each data point in the data frame
df['color']= df.apply(color, axis=1)

In [15]:
#Create the 'size' column for bubble size
from sklearn.preprocessing import MinMaxScaler
minmax_scaler=MinMaxScaler()
scaled_features=minmax_scaler.fit_transform(df[['median_sale_price_yoy']])
df['size']=pd.DataFrame(scaled_features)

#Styling changes to reduce decimal places to 2
df['avg_sale_to_list'] = pd.Series([round(val, 2) for val in df['avg_sale_to_list']], index = df.index)
df['homes_sold'] = pd.Series([round(val, 0) for val in df['homes_sold']], index = df.index)
df['median_dom'] = pd.Series([round(val, 1) for val in df['median_dom']], index = df.index)


#Styling changes to change the data format to percentages
df['median_sale_price_yoy'] = pd.Series(["{0:.1f}%".format(val * 100) for val in df['median_sale_price_yoy']], index = df.index)
df['new_listings_yoy'] = pd.Series(["{0:.1f}%".format(val * 100) for val in df['new_listings_yoy']], index = df.index) 

In [16]:
#Add 'text' column for hover-over tooltips
#You can customize what fields or information you want to show in tooltips in the code below
hover_text = []
for index, row in df.iterrows():
    hover_text.append(('Cluster Description:<br>{cluster_desc}<br><br>'+
                    'Zip Code: {Zip_Code}<br>'+
                    'Metro: {parent_metro_region}<br>'+ 
                    'Homes Sold: {homes_sold}<br>'+
                    'Median Sales Price: ${median_sale_price}<br>'+
                    'Median Sales Price YoY: {median_sale_price_yoy}<br>'+
                    'New Listings YoY: {new_listings_yoy}<br>'+
                    'Median Days on Market: {median_dom}<br>'+
                    'Avg Sales-to-Listing Price: {avg_sale_to_list}'
                    ).format(
                    cluster_desc=row['cluster_desc'],
                    Zip_Code=row['Zip_Code'],
                    parent_metro_region=row['parent_metro_region'],
                    homes_sold=row['homes_sold'],
                    median_sale_price=row['median_sale_price'],
                    median_sale_price_yoy=row['median_sale_price_yoy'],
                    new_listings_yoy=row['new_listings_yoy'],
                    median_dom=row['median_dom'],
                    avg_sale_to_list=row['avg_sale_to_list']))
df['text'] = hover_text

In [17]:
# Dictionary with dataframes for each cluster
cluster_name=df["cluster_nbr"].unique()
cluster_data = {cluster: df.loc[df["cluster_nbr"] == cluster].copy()
                            for cluster in cluster_name}

layout = go.Layout(
    title_text='VA Neighborhoods Housing Market Segments', 
    title_x=0.5,
    xaxis = go.XAxis(
        title = 'Shorter Days on Market <-------------> Longer Days on Market',
        showticklabels=False),
    yaxis = go.YAxis(
        title = 'Lower Supply <------------> Higher Supply',
        showticklabels=False
    )
)

# Create figure
fig = go.Figure(layout=layout)
for cluster_name, cluster in cluster_data.items():
    fig.add_trace(go.Scatter(
        x=cluster['PC1'], y=cluster['PC2'],
        marker = dict(color=cluster['color']),
        name=cluster_name, text=cluster['text'],
        marker_size=cluster['size']
        ))

# Tune marker appearance and layout
sizeref = 2.*max(df['size'])/(18**2) 
fig.update_traces(mode='markers', marker=dict(sizemode='area',sizeref=sizeref, line_width=2))

fig.update_layout(showlegend=False)
fig.show()

/home/awstclair/miniconda3/envs/realestate_env/lib/python3.10/site-packages/plotly/graph_objs/_deprecations.py:544: DeprecationWarning: plotly.graph_objs.XAxis is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.XAxis
  - plotly.graph_objs.layout.scene.XAxis

  warnings.warn(
/home/awstclair/miniconda3/envs/realestate_env/lib/python3.10/site-packages/plotly/graph_objs/_deprecations.py:572: DeprecationWarning: plotly.graph_objs.YAxis is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.YAxis
  - plotly.graph_objs.layout.scene.YAxis

  warnings.warn(
